In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

In [2]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

from skforecast.datasets import fetch_dataset
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries
from skforecast.model_selection_multiseries import bayesian_search_forecaster_multiseries


# Data download
# ==============================================================================
#data = fetch_dataset(name="items_sales")
#data.to_parquet('items_sales.parquet', index=True)
data = pd.read_parquet('items_sales.parquet')
data = data.asfreq('D')
data.head()
exog = pd.DataFrame({
    'exog_1': np.random.normal(loc=0, scale=1, size=data.shape[0]),
    'exog_2': np.random.normal(loc=0, scale=1, size=data.shape[0]),
    # 'exog_3': (['A'] * int(data.shape[0] / 2) + ['B'] * int(data.shape[0] / 2 + 1)),
}, index=data.index)

end_train = '2014-07-15 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test  = data.loc[end_train:, :].copy()

transformer_exog = ColumnTransformer(
                        [('scale', StandardScaler(), ['exog_1', 'exog_2']),
                        ('onehot', OneHotEncoder(), ['exog_3'])],
                        remainder = 'passthrough',
                        verbose_feature_names_out = False
                    )

# Create and train ForecasterAutoregMultiSeries
# ==============================================================================
forecaster = ForecasterAutoregMultiSeries(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1),
                 lags               = [1, 5],
                 encoding           = 'onehot',
                 transformer_series = StandardScaler(),
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = None,
                 differentiation    = None,
                 dropna_from_series = False,
                 fit_kwargs         = None,
                 forecaster_id      = None
             )

# forecaster.fit(series=data_train, exog=exog.loc[data_train.index])
forecaster.fit(series=data_train)

In [9]:
last_window, exog_values_dict, levels, prediction_index, residuals = (
    forecaster._create_predict_inputs(steps=2)
    # exog=exog.loc[data_test.index])
)

In [30]:
levels = ['item_1', 'item_2', 'item_3']
n_levels = len(levels)
lags = np.array([1, 2, 3, 4, 5])
steps = 2
lags_shape = len(lags)
exog_shape = 2
encoding = 'onehot'
series_col_names = levels
encoding_mapping = {'item_1': 0, 'item_2': 1, 'item_3': 2}
# Exog es un diccionario donde las claves son el step y los son numpy arrays en los
# que cada fila es un level y cada columna es una variable exogena.
exog = {
    1:np.full(shape=(steps, exog_shape), fill_value=99, dtype=float),
    2:np.full(shape=(steps, exog_shape), fill_value=999, dtype=float),
}
exog

{1: array([[99., 99.],
        [99., 99.]]),
 2: array([[999., 999.],
        [999., 999.]])}

In [15]:
# data = fetch_dataset(name="items_sales")
# data.to_parquet('items_sales_2.parquet', index=True)
data = pd.read_parquet("items_sales_2.parquet")
data = data.asfreq("D")
data.head()
exog = pd.DataFrame(
    {
        "exog_1": np.random.normal(loc=0, scale=1, size=data.shape[0]),
        "exog_2": np.random.normal(loc=0, scale=1, size=data.shape[0]),
    },
    index=data.index,
)

end_train = "2014-07-15 23:59:00"
data_train = data.loc[:end_train, :].copy()
data_test = data.loc[end_train:, :].copy()

forecaster.fit(series=data_train, exog=exog.loc[data_train.index])

(last_window_values_dict, exog_values_dict, levels, prediction_index, _) = (
    forecaster._create_predict_inputs(
        steps=2, levels=None, last_window=None, exog=exog.loc[data_test.index]
    )
)

In [ ]:
predictions_old = forecaster.predict(steps=50, exog=exog.loc[data_test.index])
predictions_new = forecaster.predict_new(steps=50, exog=exog.loc[data_test.index])
assert predictions_old.equals(predictions_new)

## Benchmark

In [19]:
n_series = 100
n=365
index = pd.date_range(start='2021-01-01',periods=n, freq="D")
series = [pd.Series(np.random.normal(size=n), index=index, name=f"series_{i+1}") for i in range(n_series)]
data = pd.concat(series, axis=1)
print(f"Data shape: {data.shape}")


forecaster = ForecasterAutoregMultiSeries(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1, n_estimators=20),
                 lags               = 5,
                 encoding           = 'onehot',
                 transformer_series = StandardScaler(),
                 differentiation    = 1,
             )
forecaster.fit(series = data)

Data shape: (365, 100)


In [4]:
import timeit

steps = 25

def benchmark_function():
    forecaster.predict_013(steps=steps)
times = timeit.repeat(benchmark_function, repeat=5, number=1)
times = np.array(times)
print(f"Old mean time: {times.mean()} , std: {times.std()}, max: {times.max()}, min: {times.min()}")

def benchmark_function():
    forecaster.predict(steps=steps)
times = timeit.repeat(benchmark_function, repeat=5, number=1)
times = np.array(times)
print(f"New mean time: {times.mean()} , std: {times.std()}, max: {times.max()}, min: {times.min()}")

Old mean time: 1.086991240014322 , std: 0.0704542397830175, max: 1.1698864999925718, min: 1.0002508000470698
New mean time: 0.10701592001132668 , std: 0.012142772389647675, max: 0.1298501999117434, min: 0.09339679998811334


In [6]:
assert forecaster.predict_013(steps=steps).equals(forecaster.predict(steps=steps))

In [22]:
import timeit

steps = 25
n_boot = 25

def benchmark_function():
    forecaster.predict_bootstrapping_013(steps=steps, n_boot=n_boot)
times = timeit.repeat(benchmark_function, repeat=5, number=1)
times = np.array(times)
print(f"Old mean time: {times.mean()} , std: {times.std()}, max: {times.max()}, min: {times.min()}")

def benchmark_function():
    forecaster.predict_bootstrapping(steps=steps, n_boot=n_boot)
times = timeit.repeat(benchmark_function, repeat=5, number=1)
times = np.array(times)
print(f"New mean time: {times.mean()} , std: {times.std()}, max: {times.max()}, min: {times.min()}")

Old mean time: 28.384666919987648 , std: 0.8756944847781502, max: 29.55362709995825, min: 26.888340799952857
New mean time: 0.6965285999933257 , std: 0.014021070204953256, max: 0.7125798999331892, min: 0.6794215999543667


In [21]:
new_preds = forecaster.predict_bootstrapping(steps=steps, n_boot=n_boot)
old_preds = forecaster.predict_bootstrapping_013(steps=steps, n_boot=n_boot)

for key in new_preds.keys():
    print(key)
    pd.testing.assert_frame_equal(new_preds[key], old_preds[key])

In [14]:
old_preds.head(2)

,pred_boot_0,pred_boot_1,pred_boot_2,pred_boot_3,pred_boot_4,pred_boot_5,pred_boot_6,pred_boot_7,pred_boot_8,pred_boot_9,...,pred_boot_15,pred_boot_16,pred_boot_17,pred_boot_18,pred_boot_19,pred_boot_20,pred_boot_21,pred_boot_22,pred_boot_23,pred_boot_24
2022-01-01,0.895978,0.717450,-1.011986,-0.103020,-1.293953,-1.314537,1.368333,-0.986955,0.278556,0.532839,...,-2.434767,-0.103552,1.719209,-1.330589,1.771068,-1.765523,-0.710868,0.168703,-0.127768,0.884584
2022-01-02,1.740609,0.680492,-1.235880,-0.616408,-1.458220,-1.673987,-0.482207,-0.131236,1.895295,1.231233,...,-1.978750,0.011628,0.609087,1.505434,1.179623,0.890148,-0.579240,-1.984793,0.847156,1.499289


In [24]:
%%timeit -n 2000 -r 4

array = np.full(
            shape      = (25, 500 * 10),
            fill_value = np.nan,
            dtype      = float
        )

29.2 μs ± 743 ns per loop (mean ± std. dev. of 4 runs, 2,000 loops each)


In [25]:
%%timeit -n 2000 -r 4

array = np.empty(
            shape      = (25, 500 * 10),
            dtype      = float
        )

337 ns ± 2.86 ns per loop (mean ± std. dev. of 4 runs, 2,000 loops each)


In [15]:
from sklearn.linear_model import LinearRegression

series_2 = pd.DataFrame({'1': pd.Series(np.arange(start=0, stop=50)), 
                         '2': pd.Series(np.arange(start=50, stop=100))})

In [16]:
last_window = pd.DataFrame(
                      {'1': [45, 46, 47, 48, 49], 
                       '2': [95, 96, 97, 98, 99], 
                       '3': [1, 2, 3, 4, 5]}, 
                      index = pd.RangeIndex(start=45, stop=50, step=1)
                  )

forecaster = ForecasterAutoregMultiSeries(LinearRegression(), lags=5)
forecaster.fit(series=series_2)

c:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\ForecasterAutoregMultiSeries.py:397: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 3 and the array at index 1 has size 1

In [24]:
df = forecaster._create_predict_inputs_new(steps=5, levels='1', last_window=last_window)[0]
df

,1
45,45
46,46
47,47
48,48
49,49


In [23]:
# Get column position in DataFrame using iloc
columns_to_get = ['1', '2']
df.iloc[:, df.columns.get_indexer(columns_to_get)]

,1,2
45,45,95
46,46,96
47,47,97
48,48,98
49,49,99


In [19]:
forecaster._create_predict_inputs(steps=5, levels='1', last_window=last_window)[0]

{'1': array([45, 46, 47, 48, 49])}

In [ ]:

predictions_1 = forecaster.predict(steps=5, levels='1', last_window=last_window)

In [3]:
import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.exceptions import NotFittedError
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor

# Fixtures
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import series
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog_predict

series_dict = joblib.load(r'C:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\tests\fixture_sample_multi_series.joblib')
exog_dict = joblib.load(r'C:\Users\jaesc2\GitHub\skforecast\skforecast\ForecasterAutoregMultiSeries\tests\fixture_sample_multi_series_exog.joblib')
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

In [4]:
end_train = '2003-01-30 23:59:00'

# Data scaled and differentiated
series_datetime = series.copy()
series_datetime.index = pd.date_range(start='2003-01-01', periods=len(series), freq='D')
series_dict_datetime = {
    "1": series_datetime['1'].loc[:end_train],
    "2": series_datetime['2'].loc[:end_train]
}

# Simulated exogenous variable
rng = np.random.default_rng(9876)
exog = pd.Series(
    rng.normal(loc=0, scale=1, size=len(series)), name='exog'
)
exog.index = pd.date_range(start='2003-01-01', periods=len(series), freq='D')
exog_dict_datetime = {
    "1": exog.loc[:end_train],
    "2": exog.loc[:end_train]
}
exog_pred = {
    '1': exog.loc[end_train:],
    '2': exog.loc[end_train:]
}

steps = len(series_datetime.loc[end_train:])

forecaster = ForecasterAutoregMultiSeries(
                    regressor          = LinearRegression(), 
                    lags               = 15, 
                    transformer_series = StandardScaler(),    
                    differentiation    = 1
                )
forecaster.fit(series=series_dict_datetime, exog=exog_dict_datetime)
results = forecaster._create_predict_inputs(
    steps=steps, exog=exog_pred, predict_boot=True
)

In [22]:
forecaster = ForecasterAutoregMultiSeries(
    regressor          = LGBMRegressor(
        n_estimators=2, random_state=123, verbose=-1, max_depth=2
    ),
    lags               = 5,
    encoding           = 'ordinal',
    dropna_from_series = False,
    transformer_series = StandardScaler(),
    transformer_exog   = StandardScaler(),
)
forecaster.fit(
    series=series_dict_train, exog=exog_dict_train, suppress_warnings=True
)
results = forecaster._create_predict_inputs(steps=5, exog=exog_dict_test)

c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py:936: MissingExogWarning: {'air_temperature', 'wind_speed'} not present in `exog` for series 'id_1000'. All values will be NaN. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingExogWarning)
  warnings.warn(
c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py:936: MissingExogWarning: {'cos_day_of_week'} not present in `exog` for series 'id_1003'. All values will be NaN. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingExogWarning)
  warnings.warn(
c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py:437: MissingValuesWarning: `exog` for series 'id_1000' has missing values. Most machine learning models do not allow missing values. Fitting the forecaster may fail. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingValuesWarning)
  warnings.warn(
c:\Users\jaesc2\GitHub\skforecast\skforecast\utils\utils.py:4

In [24]:
series_dict_train

{'id_1000': timestamp
 2016-01-01    1012.500694
 2016-01-02    1158.500099
 2016-01-03     983.000099
 2016-01-04    1675.750496
 2016-01-05    1586.250694
                  ...     
 2016-07-27    1344.750595
 2016-07-28    1310.000397
 2016-07-29    1297.000893
 2016-07-30    1103.500397
 2016-07-31     871.500198
 Freq: D, Name: id_1000, Length: 213, dtype: float64,
 'id_1001': timestamp
 2016-07-02    2802.000000
 2016-07-03    2807.000000
 2016-07-04    2784.000000
 2016-07-05    2755.000000
 2016-07-06    2550.000198
 2016-07-07    1876.000397
 2016-07-08    2505.000000
 2016-07-09    2276.000793
 2016-07-10    1804.000099
 2016-07-11     576.000298
 2016-07-12     799.000597
 2016-07-13     923.001390
 2016-07-14    1664.000997
 2016-07-15    1882.000496
 2016-07-16    2024.000099
 2016-07-17    1826.000099
 2016-07-18    2142.999802
 2016-07-19    2765.000000
 2016-07-20    2720.000198
 2016-07-21    2764.000000
 2016-07-22    2156.000099
 2016-07-23     825.000099
 2016-07-24

In [23]:
results[0]

,id_1000,id_1001,id_1003,id_1004
timestamp,,,,
2016-07-27,-0.352586,0.218005,-0.626520,0.614958
2016-07-28,-0.457091,0.369366,-0.746860,0.832297
2016-07-29,-0.496185,0.678968,-1.038231,0.671990
2016-07-30,-1.078102,1.163322,-0.378377,-0.243098
2016-07-31,-1.775801,1.152314,3.399801,-0.750738


In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import HistGradientBoostingRegressor

# Fixtures
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import series
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog_predict

from skforecast.utils import preprocess_y

In [16]:
forecaster = ForecasterAutoregMultiSeries(LinearRegression(), lags=3,
                                              transformer_series=StandardScaler(),
                                              differentiation=1)
forecaster.fit(series=series, exog=exog['exog_1']
            #    , store_last_window=['1']
               )

results = forecaster.predict_bootstrapping(
                steps               = 1,
                levels              = ['1', '2'],
                n_boot              = 4, 
                exog                = exog_predict['exog_1'], 
                in_sample_residuals = True
            )
results

{'1':     pred_boot_0  pred_boot_1  pred_boot_2  pred_boot_3
 50     0.479244      0.95156     0.391981     0.350606,
 '2':     pred_boot_0  pred_boot_1  pred_boot_2  pred_boot_3
 50     0.221191      0.33174     0.452264     0.191767}

In [15]:
series_2 = pd.DataFrame({'1': pd.Series(np.arange(start=0, stop=10)), 
                       '2': pd.Series(np.arange(start=50, stop=60))})

forecaster = ForecasterAutoregMultiSeries(
                    regressor          = HistGradientBoostingRegressor(
                                                  random_state         = 123
                                              ),
                    lags               = 5,
                    transformer_series = None,
                )
forecaster.fit(series=series_2)
_ = forecaster.predict_bootstrapping(steps=3, n_boot=2)
_

{'1':     pred_boot_0  pred_boot_1
 10          6.0          6.0
 11          7.0          8.0
 12          7.0          8.0,
 '2':     pred_boot_0  pred_boot_1
 10         58.0         59.0
 11         59.0         58.0
 12         56.0         57.0}

In [134]:
n_levels = len(forecaster.series_names_in_)

In [136]:
{level: _[:, i::n_levels] for i, level in enumerate(forecaster.series_names_in_)}

{'1': array([[6., 6.],
        [7., 8.],
        [7., 8.]]),
 '2': array([[58., 59.],
        [59., 58.],
        [56., 57.]])}

In [77]:
exog_cols = np.empty((3, 3), dtype=float)
exog_cols

array([[6.23042070e-307, 4.67296746e-307, 1.69121096e-306],
       [9.34596888e-307, 6.23037657e-307, 2.22526399e-307],
       [6.23053614e-307, 7.56592338e-307, 5.33590898e-322]])

In [81]:
# series = pd.DataFrame({'1': pd.Series(np.arange(start=0, stop=50)), 
#                        '2': pd.Series(np.arange(start=50, stop=100))})

df_exog = pd.DataFrame(
    {'exog_1': exog['exog_1'],
        'exog_2': ['a', 'b'] * 25,
        'exog_3': pd.Categorical(['F', 'G', 'H', 'I', 'J'] * 10)}
)

exog_predict = df_exog.copy()
exog_predict.index = pd.RangeIndex(start=50, stop=100)

categorical_features = df_exog.select_dtypes(exclude=[np.number]).columns.tolist()
transformer_exog = make_column_transformer(
                        (
                            OrdinalEncoder(
                                dtype=int,
                                handle_unknown="use_encoded_value",
                                unknown_value=-1,
                                encoded_missing_value=-1
                            ),
                            categorical_features
                        ),
                        remainder="passthrough",
                        verbose_feature_names_out=False,
                    ).set_output(transform="pandas")

forecaster = ForecasterAutoregMultiSeries(
                    regressor          = HistGradientBoostingRegressor(
                                                  categorical_features = categorical_features,
                                                  random_state         = 123
                                              ),
                    lags               = 5,
                    transformer_series = None,
                    transformer_exog   = transformer_exog
                )
forecaster.fit(series=series, exog=df_exog)

In [39]:
forecaster._recursive_predict(
                      steps            = 5,
                      levels           = levels,
                      last_window      = last_window,
                      exog_values_dict = exog_values_dict
                  )

array([[0.01772315, 1.73981623],
       [0.05236473, 1.76946476],
       [0.08680601, 1.801424  ],
       [0.12114773, 1.83453189],
       [0.15543994, 1.86821077]])

In [147]:
import re
import pytest
import numpy as np
from skforecast.preprocessing import TimeSeriesDifferentiator

# Fixtures
series = np.array([1, 4, 8, 10, 13, 22, 40, 46, 55, 70, 71], dtype=float)
y = np.array([1, 4, 8, 10, 13, 22, 40, 46], dtype=float)
next_window = np.array([55, 70, 71], dtype=float)
next_window_diff_1 = np.array([9., 15., 1.], dtype=float)
next_window_diff_2 = np.array([3., 6., -14.], dtype=float)
next_window_diff_3 = np.array([15., 3., -20.], dtype=float)

In [153]:
order = 1
next_window_diff = next_window_diff_1

transformer = TimeSeriesDifferentiator(order=order)
transformer.fit_transform(y)
print(transformer.last_values)
print(transformer.transform(y))
y_transformed = transformer.transform(y)
print(transformer.initial_values)
print(transformer.inverse_transform(y_transformed))
results = transformer.inverse_transform_next_window(next_window_diff)
results

[np.float64(46.0)]
[nan  3.  4.  2.  3.  9. 18.  6.]
[np.float64(1.0)]
[ 1.  4.  8. 10. 13. 22. 40. 46.]


array([55., 70., 71.])

In [6]:
import re
import pytest
import numpy as np
from skforecast.preprocessing import TimeSeriesDifferentiator

# Fixtures
series = np.array([1, 4, 8, 10, 13, 22, 40, 46, 55, 70, 71], dtype=float)
series_2d = np.concatenate([series.reshape(-1, 1), series.reshape(-1, 1)], axis=1)

y = np.array([1, 4, 8, 10, 13, 22, 40, 46], dtype=float)
y_2d = np.concatenate([y.reshape(-1, 1), y.reshape(-1, 1)], axis=1)

next_window = np.array([55, 70, 71], dtype=float)
next_window_diff_1 = np.array([9., 15., 1.], dtype=float)
next_window_diff_2 = np.array([3., 6., -14.], dtype=float)
next_window_diff_3 = np.array([15., 3., -20.], dtype=float)

next_window_diff_1_2d = np.concatenate([next_window_diff_1.reshape(-1, 1), next_window_diff_1.reshape(-1, 1)], axis=1)
next_window_diff_2_2d = np.concatenate([next_window_diff_2.reshape(-1, 1), next_window_diff_2.reshape(-1, 1)], axis=1)
next_window_diff_3_2d = np.concatenate([next_window_diff_3.reshape(-1, 1), next_window_diff_3.reshape(-1, 1)], axis=1)


In [10]:
order = 1
next_window_diff = next_window_diff_1_2d

order = 2
next_window_diff = next_window_diff_2_2d

# order = 3
# next_window_diff = next_window_diff_3_2d

order = 1
next_window_diff = next_window_diff_1

transformer = TimeSeriesDifferentiator(order=order)
transformer.fit_transform(y)
print(transformer.last_values)
print(transformer.transform(y))
y_transformed = transformer.transform(y)
print(transformer.initial_values)
print(transformer.inverse_transform(y_transformed))
results = transformer.inverse_transform_next_window(next_window_diff)
results

[np.float64(46.0)]
[nan  3.  4.  2.  3.  9. 18.  6.]
[np.float64(1.0)]
[ 1.  4.  8. 10. 13. 22. 40. 46.]


array([55., 70., 71.])

## Predict Bootstraping

In [74]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.exceptions import NotFittedError
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from scipy.stats import norm

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.exceptions import UnknownLevelWarning
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries

# Fixtures
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import series
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog
from skforecast.ForecasterAutoregMultiSeries.tests.fixtures_ForecasterAutoregMultiSeries import exog_predict

transformer_exog = ColumnTransformer(
                       [('scale', StandardScaler(), ['exog_1']),
                        ('onehot', OneHotEncoder(), ['exog_2'])],
                       remainder = 'passthrough',
                       verbose_feature_names_out = False
                   )

In [96]:
forecaster = ForecasterAutoregMultiSeries(
                     regressor          = LinearRegression(),
                     lags               = 3,
                     transformer_series = StandardScaler(),
                     transformer_exog   = transformer_exog,
                 )
    
forecaster.fit(series=series, exog=exog)
results = forecaster.predict_quantiles(
                steps               = 2,
                quantiles           = [0.05, 0.55, 0.95],
                levels              = '1',
                exog                = exog_predict,
                n_boot              = 4,
                in_sample_residuals = True,
                suppress_warnings   = True
            )

In [97]:
results.to_numpy()

array([[0.128375  , 0.33920342, 0.4715764 ],
       [0.09385929, 0.32079042, 0.62315962]])

In [72]:
results['2'].to_numpy()

array([[5.5061193, 3.5061193, 3.5061193, 2.5061193]])

In [73]:
results['3'].to_numpy()

array([[4.48307367, 2.48307367, 4.48307367, 4.48307367]])